# Tutorial MongoDB CRUD and Aggregation, <div style="color:red">Module Assigment 1</div>

----
student's name: Hemanthkumar Muthusamy Gunasekaran

student's email: h.muthusamy-gunasekaran@oth-aw.de

----

<br><br>


<div style="color:red">Attention, this tutorial is an Module Assigment, it is used for grade. I refere to the module course where you should hand in this notebook with your code, description and explanations</div>


<br><br>



Download the mongodb server and install it on your machine.
the notebook requires `pymongo`: `pip install pymongo`  this as well.


Mac command for running the mongoDB Server after installing the server using brew: `brew services start mongodb/brew/mongodb-community`


Make yourself familiar with the possibilities to access and wrangle information using pymongo and the underlying query mechanism of mongoDB. A document describing the aggregation pipeline of mongoDB is located in the moodle course.




#### Connection establishment

Python based Connection Management, URL of server is necessary, typically `localhost`

Database name: _`shopDB`_

Collections are referenced using the name of the collections.

Collection is created if not already existant in DB


In [5]:
!pip install pymongo


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [6]:
from pymongo import MongoClient
from bson import ObjectId
from datetime import datetime

client = MongoClient("mongodb://localhost:27017")
db = client["shopDB"]

users = db["users"]
products = db["products"]
orders = db["orders"]
reviews = db["reviews"]


# Reset
users.delete_many({})
products.delete_many({})
orders.delete_many({})
reviews.delete_many({})



DeleteResult({'n': 2, 'ok': 1.0}, acknowledged=True)

#### Database content 
some entries are added (create / inserted) to the corresponding collection

In [7]:

# Insert users
users.insert_many([
    {
        "_id": ObjectId("65f100000000000000000001"),
        "name": "Alice Müller",
        "email": "alice@example.com",
        "createdAt": datetime(2025, 1, 10, 9, 0, 0),
        "address": {"street": "Main Street 1", "city": "Berlin", "zip": "10115", "country": "DE"},
        "tags": ["premium", "newsletter"],
    },
    {
        "_id": ObjectId("65f100000000000000000002"),
        "name": "Bob Schmidt",
        "email": "bob@example.com",
        "createdAt": datetime(2025, 1, 12, 15, 30, 0),
        "address": {"street": "Oak Avenue 5", "city": "Hamburg", "zip": "20095", "country": "DE"},
        "tags": ["standard"],
    },
])

# Insert products
products.insert_many([
    {"_id": ObjectId("65f200000000000000000001"), "name": "Mechanical Keyboard",
     "category": "electronics", "price": 129.99, "stock": 42,
     "tags": ["keyboard", "mechanical", "gaming"], "createdAt": datetime(2025,1,8,10)},
    {"_id": ObjectId("65f200000000000000000002"), "name": "Noise-Cancelling Headphones",
     "category": "electronics", "price": 199.00, "stock": 15,
     "tags": ["audio","headphones"], "createdAt": datetime(2025,1,9,11,30)},
    {"_id": ObjectId("65f200000000000000000003"), "name": "Ergonomic Office Chair",
     "category": "furniture", "price": 299.00, "stock": 7,
     "tags": ["office","chair"], "createdAt": datetime(2025,1,11,13)},
])

# Insert orders
orders.insert_many([
    {
        "_id": ObjectId("65f300000000000000000001"),
        "userId": ObjectId("65f100000000000000000001"),
        "createdAt": datetime(2025,1,15,10,15),
        "status": "shipped",
        "items": [
            {"productId": ObjectId("65f200000000000000000001"), "nameSnapshot": "Mechanical Keyboard", "quantity": 1, "priceAtPurchase": 119.99},
            {"productId": ObjectId("65f200000000000000000002"), "nameSnapshot": "Noise-Cancelling Headphones", "quantity": 1, "priceAtPurchase": 189.00},
        ],
        "shippingAddress": {"street":"Main Street 1","city":"Berlin","zip":"10115","country":"DE"},
        "payment": {"method":"credit_card","paid":True,"totalAmount":308.99},
    },
    {
        "_id": ObjectId("65f300000000000000000002"),
        "userId": ObjectId("65f100000000000000000002"),
        "createdAt": datetime(2025,1,16,16,45),
        "status": "processing",
        "items": [
            {"productId": ObjectId("65f200000000000000000003"), "nameSnapshot": "Ergonomic Office Chair", "quantity": 1, 
             "priceAtPurchase": 289.00},
        ],
        "shippingAddress": {"street":"Oak Avenue 5","city":"Hamburg","zip":"20095","country":"DE"},
        "payment": {"method":"paypal","paid":False,"totalAmount":289.00},
    },
])

# Insert reviews
reviews.insert_many([
    {"_id": ObjectId("65f400000000000000000001"),
     "productId": ObjectId("65f200000000000000000001"),
     "userId": ObjectId("65f100000000000000000001"),
     "rating": 5, "title": "Great keyboard!",
     "text": "Feels amazing to type, perfect for coding.",
     "createdAt": datetime(2025,1,17,9)},
    {"_id": ObjectId("65f400000000000000000002"),
     "productId": ObjectId("65f200000000000000000002"),
     "userId": ObjectId("65f100000000000000000002"),
     "rating": 4, "title": "Good sound",
     "text": "Noise cancelling is solid, but a bit tight.",
     "createdAt": datetime(2025,1,18,14,30)},
]

)

print("Data inserted successfully")

Data inserted successfully


Adding individual Entries are also possible.

In [10]:



# 3.1 CREATE: a User
new_user = {
    "name": "Charlie Weber",
    "email": "charlie@example.com",
    "createdAt": datetime.utcnow(),
    "address": {
        "street": "Example St 10",
        "city": "Munich",
        "zip": "80331",
        "country": "DE",
    },
    "tags": ["standard"],
}
insert_result = users.insert_one(new_user)
print("Inserted new user with _id:", insert_result.inserted_id)



Inserted new user with _id: 6935caa4d9dfbf1962f43dbf


/var/folders/02/s2j6q52d0k5c5x3kf7hlyzcr0000gn/T/ipykernel_68755/2049312822.py:5: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "createdAt": datetime.utcnow(),


In [12]:

# 3.2 READ: products of categoryelectronics, price < 150
print("\nElectronics products with price < 150:")
for doc in products.find({"category": "electronics", "price": {"$lt": 150}}):
    print(doc)

# READ: Alice's Orders  
alice = users.find_one({"email": "alice@example.com"})
print("\nOrders for Alice:")
for order in orders.find({"userId": alice["_id"]}):
    print(order)

# READ: get reviews for specific product
keyboard = products.find_one({"name": "Mechanical Keyboard"})
print("\nReviews for Mechanical Keyboard:")
for review in reviews.find({"productId": keyboard["_id"]}):
    print(review)

# 3.3 UPDATE: change order status
print("\nUpdating order status for Bob's order...")
orders.update_one(
    {"_id": ObjectId("65f300000000000000000002")},
    {"$set": {"status": "shipped", "payment.paid": True}},
)
print("Updated order:\n", orders.find_one({"_id": ObjectId("65f300000000000000000002")}))

# UPDATE: Increase all elements by 10 money ;)
print("\nIncreasing price of all electronics products by 10...")
products.update_many(
    {"category": "electronics"},
    {"$inc": {"price": 10}},
)
for doc in products.find({"category": "electronics"}):
    print(doc)

# 3.4 DELETE
print("\nDeleting reviews with rating <= 2 (if any)...")
delete_result = reviews.delete_many({"rating": {"$lte": 2}})
print("Deleted count:", delete_result.deleted_count)



Electronics products with price < 150:
{'_id': ObjectId('65f200000000000000000001'), 'name': 'Mechanical Keyboard', 'category': 'electronics', 'price': 129.99, 'stock': 42, 'tags': ['keyboard', 'mechanical', 'gaming'], 'createdAt': datetime.datetime(2025, 1, 8, 10, 0)}

Orders for Alice:
{'_id': ObjectId('65f300000000000000000001'), 'userId': ObjectId('65f100000000000000000001'), 'createdAt': datetime.datetime(2025, 1, 15, 10, 15), 'status': 'shipped', 'items': [{'productId': ObjectId('65f200000000000000000001'), 'nameSnapshot': 'Mechanical Keyboard', 'quantity': 1, 'priceAtPurchase': 119.99}, {'productId': ObjectId('65f200000000000000000002'), 'nameSnapshot': 'Noise-Cancelling Headphones', 'quantity': 1, 'priceAtPurchase': 189.0}], 'shippingAddress': {'street': 'Main Street 1', 'city': 'Berlin', 'zip': '10115', 'country': 'DE'}, 'payment': {'method': 'credit_card', 'paid': True, 'totalAmount': 308.99}}

Reviews for Mechanical Keyboard:
{'_id': ObjectId('65f400000000000000000001'), 'p

# Task (MongoDB,  Nested Documents)

<div style="color:orange">Create for each task a specific python function (<code>taks1(...), task2(...), task3(...)</code>)!</div> 

<br>

<div style="color:red">Make also sure to compare your solution against the solution provided by chatgpt. Discuss overlaps or divergences of the solutions.</div>

In addition to the implementation tasks, discuss where atomicity and multi-document transactions are required in this scenario (user creation, stock validation and update, order insertion).
Briefly explain the potential consequences if no atomicity / transaction support is used (e.g. race conditions, overselling, inconsistent orders).
Finally, provide a concrete solution that uses MongoDB transactions (via pymongo and sessions) to ensure that user handling, stock updates, and order creation are executed atomically.

<br>

**Be aware to combine all subTask Functions in the last function**

Implement python code to create a **new order in MongoDB** with the following requirements:


1. **User handling (embedded user document in the order)**  <code>taks1(...)</code>
   - The order is placed by a user identified by some unique field (e.g. `email`).  
   - Check in the MongoDB `users` collection whether this user already exists.  
   - If the user does **not** exist, insert a **new user document** into the `users` collection. Align the format of the new document with the previously stored documents.
   - In the order document, embed a **nested `user` subdocument** (e.g. containing `userId`, `name`, `email`, etc.), so that user information is directly available inside the order.


In [13]:
def task1(db, session, email, user_data_if_new):
    users_col = db["users"]
    
    # Check if user exists
    user = users_col.find_one({"email": email}, session=session)
    
    # Create if missing
    if not user:
        # Ensure the new data has the required timestamp if not provided
        if "createdAt" not in user_data_if_new:
            user_data_if_new["createdAt"] = datetime.now()
            
        result = users_col.insert_one(user_data_if_new, session=session)
        
        # Fetch the complete document we just inserted to get the _id
        user = users_col.find_one({"_id": result.inserted_id}, session=session)

    # Create the embedded document snapshot
    # We explicitly select fields to snapshot so the order has a permanent record of user details
    embedded_user = {
        "userId": user["_id"],
        "name": user["name"],
        "email": user["email"],
        "address": user["address"]
    }
    
    return embedded_user

### Task 1: User Handling

**Explanation:**
To handle the user part of the order, I first check the `users` collection to see if a user with the given email already exists. If they don't, I insert the new user data right away so every order is linked to a real profile. Finally, instead of just passing back the whole database document, I create a specific "snapshot" containing just the ID, name, email, and address. This ensures the order keeps a record of the address *as it was at the time of purchase*, even if the user updates it later.


2. **Order content with embedded product descriptions**  <code>taks2(...)</code>
   - The order must contain **at least one item**.
   - Each item refers to a product stored in the `products` collection (e.g. via `productId`).  
   - For each item, fetch the corresponding product from `products` and embed a **nested product description** inside the order, for example:
     ```json
     {
       "productId": "...",
       "name": "...",
       "priceAtPurchase": ...,
       "quantity": ...
     }
     ```
   - This way, the order stores a **snapshot of the product data** at the time of purchase.


In [14]:
def task2(db, session, items_to_order):
    products_col = db["products"]
    order_items = []
    
    for item in items_to_order:
        product_id = item["productId"]
        quantity = item["quantity"]
        
        # Fetch the current product details
        product = products_col.find_one({"_id": product_id}, session=session)
        
        if not product:
            # In a real app, you might raise an error here
            print(f"Warning: Product {product_id} not found!")
            continue
            
        # Create the snapshot
        # We lock in the price and name NOW, so future changes don't affect this order
        snapshot = {
            "productId": product["_id"],
            "name": product["name"],  # Snapshotting the name
            "priceAtPurchase": product["price"], # Snapshotting the price
            "quantity": quantity
        }
        
        order_items.append(snapshot)
        
    return order_items

### Task 2: Order Content (Product Snapshots)

**Explanation:**
For the order content, I needed to make sure the price and name of a product are "locked in" at the moment of purchase. I iterated through the list of items the user wanted to buy and fetched their current details from the `products` collection. Then, I created a list of snapshots that includes the `name` and `priceAtPurchase` right inside the order. This way, if the shop changes the price of an item next week, the old order history remains accurate and doesn't change.


3. **Stock validation and update in MongoDB**  
   - Before inserting the order into the `orders` collection:
     - For each item, query the `products` collection and check whether there is **sufficient `stock`** available.  
     - If **any** product does not have enough stock, **abort** the operation (do not insert the order, do not modify stock).  
     - If all products have sufficient stock, **decrease the stock** in `products` for each item (e.g. using `$inc: { stock: -quantity }`).




In [15]:
def task3(db, session, order_items):
    products_col = db["products"]
    
    for item in order_items:
        product_id = item["productId"]
        quantity = item["quantity"]
        
        # ATOMIC CHECK & UPDATE
        # We try to find the product AND ensure it has enough stock in one query.
        # If stock < quantity, this update will simply do nothing (modified_count = 0).
        result = products_col.update_one(
            {
                "_id": product_id, 
                "stock": {"$gte": quantity}  # The validation condition
            },
            {
                "$inc": {"stock": -quantity} # The update action
            },
            session=session
        )
        
        # If the DB didn't update anything, it means stock was insufficient
        if result.modified_count == 0:
            raise ValueError(f"Insufficient stock for product '{item['name']}' (ID: {product_id})")
            
    print("Stock validation passed and updates applied.")

### Task 3: Stock Validation and Update

**Explanation:**
This function handles the critical job of inventory management. I iterated through every item in the proposed order and performed an "atomic conditional update." This means I asked MongoDB to find the product **and** check if the stock level was high enough in a single step. If the stock was sufficient, I immediately decreased it using `$inc`. If the stock wasn't high enough, the update failed (returning a count of 0), and I raised an error. In our transaction system, this error acts as an emergency stop, canceling the whole order so we don't sell items we don't have.

4. **Order creation in the `orders` collection**  
   - Only after:
     - (a) the user has been ensured to exist (and created if necessary), and  
     - (b) all stock levels have been validated and updated,  
   - insert a new document into the `orders` collection.  
   - The order document should contain:
     - an `_id` or `orderId`,  
     - a `createdAt` timestamp,  
     - the embedded `user` subdocument,  
     - an `items` array containing the embedded product descriptions with quantities,  
     - optional payment/total fields (e.g. `totalAmount`).


In [16]:
def task4(db, session, user_doc, order_items):
    orders_col = db["orders"]
    
    # Calculate the total amount for the record
    total_amount = 0.0
    for item in order_items:
        total_amount += item["priceAtPurchase"] * item["quantity"]
    
    # Construct the final order document
    new_order = {
        "userId": user_doc["userId"], # Reference for searching
        "createdAt": datetime.now(),
        "status": "processing",
        "user": user_doc,             # The Snapshot from Task 1
        "items": order_items,         # The Snapshots from Task 2
        "payment": {
            "totalAmount": round(total_amount, 2),
            "method": "credit_card",  # Default for this example
            "paid": True
        }
    }
    
    # Save to database
    result = orders_col.insert_one(new_order, session=session)
    
    return result.inserted_id

### Task 4: Order Creation

**Explanation:**
This function is the final step where we actually save the order. First, I calculated the total cost by adding up the price of all the items. Then, I built the final order document by putting together the user snapshot (from Task 1) and the list of product snapshots (from Task 2). Finally, I inserted this complete document into the `orders` collection. Since this is part of our transaction, it only gets saved permanently if every other part of the process succeeds.

In [17]:
def combinationOfAllTasks(client, db, user_email, new_user_data, items_to_order):
    """
    Executes the entire order workflow within an ACID transaction.
    """
    # 1. Start the Session
    with client.start_session() as session:
        # 2. Start the Transaction
        # Everything inside this 'with' block is atomic.
        # If an error is raised, it auto-aborts (rollbacks).
        with session.start_transaction():
            try:
                print("Starting Transaction...")
                
                # Step A: Ensure User Exists
                user_doc = task1(db, session, user_email, new_user_data)
                print(f"User processed: {user_doc['name']}")
                
                # Step B: Prepare Product Snapshots
                order_items = task2(db, session, items_to_order)
                print(f"Prepared {len(order_items)} items for order.")
                
                # Step C: Validate and Update Stock
                # This will RAISE an exception if stock is low, triggering a rollback
                task3(db, session, order_items)
                
                # Step D: Create the Order
                order_id = task4(db, session, user_doc, order_items)
                
                print(f"Transaction Committed. Order ID: {order_id}")
                return order_id

            except Exception as e:
                # If any step failed, we print the error.
                # The 'with' block automatically handles the abort/rollback.
                print(f"Transaction Aborted! Reason: {e}")
                raise e

### Transaction Logic (Combining All Tasks)

**Explanation:**
This function acts as the manager that ties everything together using a "Multi-Document Transaction." I started a session and wrapped all the previous tasks (user creation, stock updates, order creation) inside a single transaction block. This ensures that these actions happen as one indivisible unit—either they *all* succeed, or *none* of them happen. If the stock check fails or any other error occurs, the system automatically "rolls back" everything, so we never end up with a broken state like money taken but no order created.

### Discussion: Why We Need Atomicity

**1. Where Atomicity is Required**
In this scenario, "placing an order" looks like one action to the user, but under the hood, it actually requires writing data to three different places:
* **Users Collection:** To register new customers.
* **Products Collection:** To update inventory levels.
* **Orders Collection:** To save the final receipt.

Because these are separate steps, we need **Atomicity** to treat them as a single, all-or-nothing unit.

**2. The Risks of No Atomicity**
If we ran these steps independently without transactions, two major problems could occur:
* **Race Conditions (Overselling):** If two customers try to buy the last item at the exact same moment, both might read the stock as "1" before the update happens. Both would proceed to checkout, and we would end up selling stock we don't have (negative inventory).
* **Data Inconsistency:** If the script crashes halfway through—for example, after the stock is reduced but *before* the order is saved—the database breaks. We would have missing inventory with no record of who bought it, or new users created who never actually ordered anything.

**3. Our Solution Strategy**
To fix this, I used **MongoDB’s Multi-Document ACID Transactions**. 
* I initiated a session using `client.start_session()` and wrapped the entire logic in `session.start_transaction()`.
* Crucially, I passed this `session` to every database operation (`insert`, `update`).
* This setup ensures safety: if any part of the process fails (like a stock check), the transaction automatically aborts and "rolls back" every change, leaving the database exactly as it was before we started.

## Aggregations in MongoDB

We now want to apply specific aggregation operations to retrieve combined information from useser and their orders. The json file contains a larger dataset which has to be analyzed.

In [ ]:
import json
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017")
db = client["shopOrders"]

with open("orders.json", encoding="utf-8") as f:
    data = json.load(f)

db.users.insert_many(data["users"])
db.products.insert_many(data["products"])
db.orders.insert_many(data["orders"])
db.reviews.insert_many(data["reviews"])

Example pipeline for getting mean information for order values:

In [ ]:
pipeline = [
    {
        "$group": {
            "_id": None,
            "avgOrderValue": {"$avg": "$payment.totalAmount"},
            "minOrderValue": {"$min": "$payment.totalAmount"},
            "maxOrderValue": {"$max": "$payment.totalAmount"},
            "countOrders": {"$sum": 1},
        }
    }
]

list(db.orders.aggregate(pipeline))

### Get all Users, who ordered more than twice of the avarage price of an order

In [ ]:
pipeline = [
    # 1: Durchschnitt berechnen
    {
        "$group": {
            "_id": None,
            "avgOrderValue": { "$avg": "$payment.totalAmount" }
        }
    },

    # 2: Den Durchschnittswert in die nächste Pipelinephase übertragen
    {
        "$lookup": {
            "from": "orders",
            "pipeline": [
                {
                    "$match": {
                        # Filter: Bestellungen > 2 × Durchschnitt
                        "$expr": {
                            "$gt": [
                                "$payment.totalAmount",
                                { "$multiply": [ "$$avgValue", 2 ] }
                            ]
                        }
                    }
                },
                { "$group": { "_id": "$userId" } }
            ],
            "as": "usersAboveAverage",
            "let": { "avgValue": "$avgOrderValue" }
        }
    },

    # 3: Auflösen
    { "$unwind": "$usersAboveAverage" },

    # 4: Nutzerinformationen anhängen
    {
        "$lookup": {
            "from": "users",
            "localField": "usersAboveAverage._id",
            "foreignField": "_id",
            "as": "user"
        }
    },
    { "$unwind": "$user" },

    # 5: Ausgabe formatieren
    {
        "$project": {
            "_id": "$user._id",
            "name": "$user.name",
            "email": "$user.email"
        }
    }
]

result = list(db.orders.aggregate(pipeline))
result



## Task 1 – Count Orders per User
For each user, determine how many orders they have placed. Sort the results in descending order based on the order count.

**Expected output fields:**
- `userId`
- `name`
- `orderCount`

---

In [19]:
import json
from pymongo import MongoClient

# 1. Connect to the Server
client = MongoClient("mongodb://localhost:27017")

# 2. Define the db_analytics variable (Connecting to 'shopOrders')
db_analytics = client["shopOrders"]

# 3. Reload the data (Optional but safe to ensure it exists)
# Note: If you already loaded it successfully before, you technically only need lines 1-2.
# But running this ensures you are starting fresh.
db_analytics.users.delete_many({})
db_analytics.products.delete_many({})
db_analytics.orders.delete_many({})
db_analytics.reviews.delete_many({})

with open("orders.json", encoding="utf-8") as f:
    data = json.load(f)

db_analytics.users.insert_many(data["users"])
db_analytics.products.insert_many(data["products"])
db_analytics.orders.insert_many(data["orders"])
db_analytics.reviews.insert_many(data["reviews"])

print("Setup Complete: 'db_analytics' is now defined.")

Setup Complete: 'db_analytics' is now defined.


In [20]:
# Aggregation Pipeline for Task 1
pipeline = [
    # 1. Group by userId and count the orders
    {
        "$group": {
            "_id": "$userId",
            "orderCount": {"$sum": 1}
        }
    },
    # 2. Join with the 'users' collection to get the name
    {
        "$lookup": {
            "from": "users",
            "localField": "_id",
            "foreignField": "_id",
            "as": "userInfo"
        }
    },
    # 3. Flatten the userInfo array (lookup returns a list)
    {
        "$unwind": "$userInfo"
    },
    # 4. Sort by orderCount (descending)
    {
        "$sort": {"orderCount": -1}
    },
    # 5. Format the output to match requirements
    {
        "$project": {
            "_id": 0,
            "userId": "$_id",
            "name": "$userInfo.name",
            "orderCount": 1
        }
    }
]

# Execute and print
results = list(db_analytics.orders.aggregate(pipeline))

print(f"Found {len(results)} users with orders.")
for user in results:
    print(user)

Found 59 users with orders.
{'orderCount': 11, 'userId': '65f100000000000000000057', 'name': 'Eva Hoffmann'}
{'orderCount': 10, 'userId': '65f100000000000000000051', 'name': 'Ines Wagner'}
{'orderCount': 10, 'userId': '65f100000000000000000036', 'name': 'Ines Schröder'}
{'orderCount': 9, 'userId': '65f100000000000000000019', 'name': 'Eva Becker'}
{'orderCount': 8, 'userId': '65f100000000000000000021', 'name': 'Charlie Schmidt'}
{'orderCount': 8, 'userId': '65f100000000000000000025', 'name': 'Alice Hoffmann'}
{'orderCount': 7, 'userId': '65f100000000000000000042', 'name': 'Olivia Fischer'}
{'orderCount': 7, 'userId': '65f100000000000000000055', 'name': 'Charlie Becker'}
{'orderCount': 7, 'userId': '65f100000000000000000008', 'name': 'Lukas Hoffmann'}
{'orderCount': 6, 'userId': '65f100000000000000000027', 'name': 'Lukas Meyer'}
{'orderCount': 6, 'userId': '65f100000000000000000037', 'name': 'Hannes Schröder'}
{'orderCount': 6, 'userId': '65f100000000000000000020', 'name': 'Stefan Richte

Task 1: Count Orders per User

Explanation: To find out who orders the most, I started by grouping the orders by userId and counting them. Since the order documents only have the user's ID, I had to use $lookup to join with the users collection and grab the actual names. Finally, I sorted the results in descending order so the most active users appear at the top of the list.

## Task 2 – Average Order Value
Compute the average total order value across all orders.
Additionally, include the minimum and maximum order value found.

In [21]:
# Pipeline for Task 2
pipeline = [
    {
        "$group": {
            # _id: None means "Group everything into one single bucket"
            "_id": None,
            "averageValue": {"$avg": "$payment.totalAmount"},
            "minValue": {"$min": "$payment.totalAmount"},
            "maxValue": {"$max": "$payment.totalAmount"}
        }
    }
]

# Run the aggregation
stats = list(db_analytics.orders.aggregate(pipeline))

if stats:
    # formatting the output to look nice
    data = stats[0]
    print(f"--- Order Statistics ---")
    print(f"Average Order Value: {data['averageValue']:.2f} €")
    print(f"Lowest Order Value:  {data['minValue']:.2f} €")
    print(f"Highest Order Value: {data['maxValue']:.2f} €")
else:
    print("No orders found to analyze.")

--- Order Statistics ---
Average Order Value: 868.71 €
Lowest Order Value:  11.53 €
Highest Order Value: 3691.32 €


### Task 2: Average Order Value

**Explanation:**
For this task, I needed statistics for the entire store, not specific users. By setting the group `_id` to `None`, I told MongoDB to treat the entire `orders` collection as one single group. Then, I simply applied the `$avg`, `$min`, and `$max` operators to the `totalAmount` field to get the global figures.


## Task 3 – Product Revenue Ranking
For each product, calculate the total revenue generated.

**Definition:**
```
revenue = SUM(quantity × priceAtPurchase)
```

Sort products by total revenue in descending order.

**Expected output fields:**

- `productId`
- `name`
- `totalRevenue`

---

In [22]:
# Pipeline for Task 3
pipeline = [
    # 1. Break the 'items' array into individual documents
    # Before: One order with 3 items
    # After:  3 documents, each with one item
    {
        "$unwind": "$items"
    },
    # 2. Calculate revenue per product
    # We group by the productId found inside the item
    {
        "$group": {
            "_id": "$items.productId",
            "totalRevenue": {
                "$sum": {
                    "$multiply": ["$items.quantity", "$items.priceAtPurchase"]
                }
            }
        }
    },
    # 3. Get the product name (Join with products collection)
    {
        "$lookup": {
            "from": "products",
            "localField": "_id",
            "foreignField": "_id",
            "as": "productDetails"
        }
    },
    # 4. Flatten the lookup result array
    {
        "$unwind": "$productDetails"
    },
    # 5. Sort by highest revenue first
    {
        "$sort": {"totalRevenue": -1}
    },
    # 6. Format the output
    {
        "$project": {
            "_id": 0,
            "productId": "$_id",
            "name": "$productDetails.name",
            "totalRevenue": 1
        }
    }
]

# Run the pipeline
revenue_ranking = list(db_analytics.orders.aggregate(pipeline))

print(f"--- Top 5 Products by Revenue ---")
for prod in revenue_ranking[:5]:
    print(f"{prod['name']}: {prod['totalRevenue']:.2f} €")

--- Top 5 Products by Revenue ---
Desk Lamp: 20776.82 €
USB-C Hub: 18518.41 €
Wireless Mouse: 17331.17 €
Desk Lamp: 17324.11 €
Wireless Mouse: 13530.81 €


### Task 3: Product Revenue Ranking

**Explanation:**
Since products are stored inside an array within each order, I couldn't sum up the revenue immediately. I used `$unwind` first to break the arrays apart into individual item documents. After that, I calculated the total revenue for each item (`quantity` × `price`) and grouped them by `productId`. A final lookup added the readable product names before sorting them by highest revenue.

## Task 4 – User Classification by Spending Behavior
Classify each user based on their *average* order value.

**Categories:**
- `"low"`: < 100 EUR
- `"medium"`: 100–300 EUR
- `"high"`: > 300 EUR

**Expected output fields:**
- `_id`
- `name`
- `avgOrderValue`
- `category`

---

Attention - possible you have to adjust the category boundaries.


In [23]:
# Pipeline for Task 4
pipeline = [
    # 1. Calculate the average order value per user
    {
        "$group": {
            "_id": "$userId",
            "avgOrderValue": {"$avg": "$payment.totalAmount"}
        }
    },
    # 2. Get the user's name
    {
        "$lookup": {
            "from": "users",
            "localField": "_id",
            "foreignField": "_id",
            "as": "userInfo"
        }
    },
    {
        "$unwind": "$userInfo"
    },
    # 3. Classify the user based on the calculated average
    {
        "$project": {
            "_id": 1,
            "name": "$userInfo.name",
            "avgOrderValue": 1,
            "category": {
                "$switch": {
                    "branches": [
                        # Case 1: Less than 100
                        {"case": {"$lt": ["$avgOrderValue", 100]}, "then": "low"},
                        # Case 2: Less than 300 (Technically 100-300, since <100 is already caught)
                        {"case": {"$lt": ["$avgOrderValue", 300]}, "then": "medium"}
                    ],
                    # Case 3: Everything else (Greater than 300)
                    "default": "high"
                }
            }
        }
    }
]

# Execute
user_categories = list(db_analytics.orders.aggregate(pipeline))

print(f"--- User Classifications (First 5) ---")
for user in user_categories[:5]:
    print(f"User: {user['name']}")
    print(f"  Avg Spend: {user['avgOrderValue']:.2f} €")
    print(f"  Category:  {user['category']}")
    print("-" * 30)

--- User Classifications (First 5) ---
User: Zoe Schäfer
  Avg Spend: 90.31 €
  Category:  low
------------------------------
User: Charlie Becker
  Avg Spend: 1491.02 €
  Category:  high
------------------------------
User: Lukas Hoffmann
  Avg Spend: 1139.65 €
  Category:  high
------------------------------
User: Mara Hoffmann
  Avg Spend: 673.25 €
  Category:  high
------------------------------
User: Yann Schröder
  Avg Spend: 1199.41 €
  Category:  high
------------------------------


### Task 4: User Classification

**Explanation:**
The goal here was to tag users based on their spending habits. First, I grouped by `userId` to calculate their average order value. Then, inside the `$project` stage, I used a `$switch` operator (which works like an if-else statement) to assign a category: "low" for under 100€, "medium" for up to 300€, and "high" for anything above that.

## Task 5 – Top 3 Users per Product Category by Spending
For each product category, determine the top 3 users ranked by total spending within that category.

**Rules:**
1. Join orders with products to determine the categories involved.
2. Compute the total spending per user per category
   (`quantity × priceAtPurchase` summed across all orders).
3. For each category, return the top three highest-spending users.

**Expected output structure example:**
```json
{
  "category": "electronics",
  "topUsers": [
    { "userId": "...", "name": "...", "totalSpent": 542.90 },
    { "userId": "...", "name": "...", "totalSpent": 301.50 },
    { "userId": "...", "name": "...", "totalSpent": 199.99 }
  ]
}
```


In [24]:
# Pipeline for Task 5
pipeline = [
    # 1. Break down orders into individual items
    { "$unwind": "$items" },

    # 2. Get the category for each item (Join with products)
    {
        "$lookup": {
            "from": "products",
            "localField": "items.productId",
            "foreignField": "_id",
            "as": "productInfo"
        }
    },
    { "$unwind": "$productInfo" },

    # 3. Calculate spend per User per Category
    # We group by *both* Category and UserId to get individual user totals
    {
        "$group": {
            "_id": {
                "category": "$productInfo.category",
                "userId": "$userId"
            },
            "totalSpent": {
                "$sum": { "$multiply": ["$items.quantity", "$items.priceAtPurchase"] }
            }
        }
    },

    # 4. Sort by highest spend first
    # This ensures that when we group them next, the big spenders are at the top
    { "$sort": { "totalSpent": -1 } },

    # 5. Get the User's Name (Join with users)
    {
        "$lookup": {
            "from": "users",
            "localField": "_id.userId",
            "foreignField": "_id",
            "as": "userInfo"
        }
    },
    { "$unwind": "$userInfo" },

    # 6. Group by Category and collect the top 3 users
    {
        "$group": {
            "_id": "$_id.category",
            "topUsers": {
                "$push": {
                    "userId": "$_id.userId",
                    "name": "$userInfo.name",
                    "totalSpent": "$totalSpent"
                }
            }
        }
    },

    # 7. Keep only the top 3 and format the output
    {
        "$project": {
            "_id": 0,
            "category": "$_id",
            "topUsers": { "$slice": ["$topUsers", 3] } # The 'Limit' trick for arrays
        }
    },
    
    # Optional: Sort categories alphabetically for cleaner output
    { "$sort": { "category": 1 } }
]

# Execute
category_rankings = list(db_analytics.orders.aggregate(pipeline))

# Print Results
for cat_data in category_rankings:
    print(f"--- Category: {cat_data['category'].upper()} ---")
    for i, user in enumerate(cat_data['topUsers'], 1):
        print(f"{i}. {user['name']}: {user['totalSpent']:.2f} €")
    print("")

--- Category: BOOKS ---
1. Eva Becker: 799.49 €
2. Eva Hoffmann: 573.17 €
3. Charlie Klein: 554.81 €

--- Category: ELECTRONICS ---
1. Charlie Klein: 4262.75 €
2. Lukas Hoffmann: 4152.11 €
3. Olivia Fischer: 3599.29 €

--- Category: FURNITURE ---
1. Eva Hoffmann: 4187.32 €
2. Eva Becker: 4166.81 €
3. Alice Becker: 3442.39 €

--- Category: GAMING ---
1. Charlie Schmidt: 1377.83 €
2. Hannes Wagner: 1157.23 €
3. Yann Schröder: 896.82 €

--- Category: KITCHEN ---
1. Ines Schröder: 4189.67 €
2. Eva Hoffmann: 2046.76 €
3. Hannes Wolf: 2013.45 €

--- Category: OFFICE ---
1. Eva Hoffmann: 272.73 €
2. Alice Hoffmann: 268.62 €
3. Ines Wagner: 232.35 €



### Task 5: Top 3 Users per Category

**Explanation:**
This required a few steps because the category info isn't in the order itself.
1. I unwound the items and looked up the `products` collection to get the category for every item sold.
2. I grouped the data by **both** `category` and `userId` to calculate exactly how much each person spent in each specific category.
3. After sorting these results to put the big spenders on top, I grouped by category again and used `$slice` to keep only the top 3 users for each one.